<a href="https://colab.research.google.com/github/danielsaggau/deep_unsupervised_learning/blob/main/replication_delorian.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [5]:
%cd /content/gdrive/MyDrive/unsup_gen_for_cms_reasoning


/content/gdrive/MyDrive/unsup_gen_for_cms_reasoning


In [ ]:
!pip install -r requirements.txt

In [4]:
!sh run_counterfactual_main.sh

Streaming output truncated to the last 5000 lines.
-------Iteration-------  12
loss: 0.0000
[Backward]:   After a few minutes, the lights came back on.

-------Iteration-------  13
loss: 0.0000
[Backward]:   After a few minutes, the lights came back on.

-------Iteration-------  14
loss: 0.0000
[Backward]:   After a few minutes, the lights came back on.

-------Iteration-------  15
loss: 0.0000
[Backward]:   After a few minutes, the lights came back on.
[Forward]:   They began to play, but the tornado destroyed the building.
 70% 7/10 [00:08<00:03,  1.24s/it]
Pass  7

-------Iteration-------  1
loss: 6.8444
[Backward]:   After a playing sing, the lights came back on.

-------Iteration-------  2
loss: 1.0755
[Backward]:   After a few minutes, the lights came back on.

-------Iteration-------  3
loss: 0.0051
[Backward]:   After a few minutes, the lights came back on.

-------Iteration-------  4
loss: 0.0001
[Backward]:   After a few minutes, the lights came back on.

-------Iteration----

In [ ]:
%cd /content/gdrive/MyDrive/unsup_gen_for_cms_reasoning
!sh run_abductive_main.sh

/content/gdrive/MyDrive/unsup_gen_for_cms_reasoning
100% 20/20 [11:38<00:00, 34.90s/it]
100% 20/20 [11:36<00:00, 34.83s/it]
100% 20/20 [10:29<00:00, 31.50s/it]
100% 20/20 [10:57<00:00, 32.88s/it]
 15% 3/20 [01:42<09:47, 34.53s/it]

In [14]:
%cd /content/gdrive/MyDrive/unsup_gen_for_cms_reasoning/ranking

/content/gdrive/MyDrive/unsup_gen_for_cms_reasoning/ranking


In [20]:
!sh 'run_counterfactual_ranking.sh'

Reading  /content/gdrive/MyDrive/unsup_gen_for_cms_reasoning/output/counterfactual/counterfactual_output_np5_nbi5.json
Reading  /content/gdrive/MyDrive/unsup_gen_for_cms_reasoning/output/counterfactual/counterfactual_output_np5_nbi8.json
Reading  /content/gdrive/MyDrive/unsup_gen_for_cms_reasoning/output/counterfactual/counterfactual_output_np5_nbi10.json
Reading  /content/gdrive/MyDrive/unsup_gen_for_cms_reasoning/output/counterfactual/counterfactual_output_np5_nbi15.json
Reading  /content/gdrive/MyDrive/unsup_gen_for_cms_reasoning/output/counterfactual/counterfactual_output_np10_nbi5.json
Reading  /content/gdrive/MyDrive/unsup_gen_for_cms_reasoning/output/counterfactual/counterfactual_output_np10_nbi8.json
Reading  /content/gdrive/MyDrive/unsup_gen_for_cms_reasoning/output/counterfactual/counterfactual_output_np10_nbi10.json
Reading  /content/gdrive/MyDrive/unsup_gen_for_cms_reasoning/output/counterfactual/counterfactual_output_np10_nbi15.json
Original data size:  1871
Downloading: 1

In [3]:
import json

In [ ]:
with open('/content/gdrive/MyDrive/unsup_gen_for_cms_reasoning/data/abductive/test-w-comet-preds.jsonl', 'r') as json_file:
    json_list = list(json_file)

for json_str in json_list:
    result = json.loads(json_str)
    print(f"result: {result}")
    print(isinstance(result, dict))

result

In [ ]:
!sh ranking/run_abductive_ranking.sh

/content/unsup_gen_for_cms_reasoning/ranking
2021-05-20 17:32:52.056172: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
Original data size:  2
Downloading: 100% 433/433 [00:00<00:00, 325kB/s]
Downloading: 100% 436M/436M [00:33<00:00, 12.9MB/s]
Downloading: 100% 213k/213k [00:00<00:00, 293kB/s] 
printing ..//output/abductive/ranking//o1o2.txt
printing ..//output/abductive/ranking//o1ho2.txt
printing ..//output/abductive/ranking//hypotheses.txt
printing ..//output/abductive/ranking//ranked_hypotheses.txt
printing ..//output/abductive/ranking//bert_score.txt


In [ ]:
!pip install datasets
from datasets import load_dataset
dataset = load_dataset('art',data_dir="/content/unsup_gen_for_cms_reasoning/data/abductive" )

Using custom data configuration default-fb72da50753a83a2


0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset art downloaded and prepared to /root/.cache/huggingface/datasets/art/default-fb72da50753a83a2/0.1.0/e4b20acfcea873d587a87e817a63c02ce080bce28cd4c322dbd476fd07286b49. Subsequent calls will reuse this data.


In [ ]:
dataset['train'][50000]

{'hypothesis_1': 'It took 10 minutes for the snow plow to come through.',
 'hypothesis_2': 'It took 10 days for the snow plow to come through.',
 'label': 2,
 'observation_1': 'There was ten feet of snow outside.',
 'observation_2': 'In all that time I was unable to check my mail.'}

In [ ]:
%cd /content/unsup_gen_for_cms_reasoning
!sh /content/unsup_gen_for_cms_reasoning/run_abductive_main.sh

/content/unsup_gen_for_cms_reasoning
Traceback (most recent call last):
  File "abductive_main.py", line 495, in <module>
    records = read_inputs(args.input_file)
  File "abductive_main.py", line 36, in read_inputs
    with open(input_file) as f:
FileNotFoundError: [Errno 2] No such file or directory: 'dataset'
